In [ ]:
!pip install seaborn --upgrade

In [ ]:
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from tensorflow.keras import models, layers
from tensorflow.keras.callbacks import EarlyStopping

# PIMA Indians Diabetes prediction

## Data

In [ ]:
df = pd.read_csv('/kaggle/input/pima-indians-diabetes-database/diabetes.csv')
df.head()

In [ ]:
df.info()

In [ ]:
plt.figure(figsize=(10,6))

heatmap = sns.heatmap(df.corr(), vmin=-1,vmax=1, annot=True, cmap='viridis')

heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':12}, pad=12)
plt.show()

In [ ]:
fig, ax=plt.subplots(7,1,figsize=(10,45))

sns.histplot(ax=ax[0],data=df, x='Age',hue='Outcome',kde=True)
sns.histplot(ax=ax[1],data=df, x='Glucose',hue='Outcome',kde=True)
sns.histplot(ax=ax[2],data=df, x='Insulin',hue='Outcome',kde=True)
sns.histplot(ax=ax[3],data=df, x='BMI',hue='Outcome',kde=True)
sns.histplot(ax=ax[4],data=df, x='BloodPressure',hue='Outcome',kde=True)
sns.histplot(ax=ax[5],data=df, x='SkinThickness',hue='Outcome',kde=True)
sns.histplot(ax=ax[6],data=df, x='DiabetesPedigreeFunction',hue='Outcome',kde=True)

plt.show()

Future Scaling:
* Normalize for :
    * Age
    * Insulin
    * Diabetes Pedigree Function
* Standarize for :
    * Glucose
    * BMI
    * Blood Pressure
    * Skin Thickness

## Model

### Train Test Split

In [ ]:
X_tr, X_ts, Y_tr, Y_ts = train_test_split(df.drop(columns='Outcome'), df['Outcome'],
                                          test_size=0.15, shuffle=True,random_state=42,stratify=df['Outcome'])

scaler1 = MinMaxScaler()
scaler2 = StandardScaler()
X_tr[['Age','Insulin','DiabetesPedigreeFunction']] = scaler1.fit_transform(X_tr[['Age','Insulin','DiabetesPedigreeFunction']])
X_ts[['Age','Insulin','DiabetesPedigreeFunction']] = scaler1.transform(X_ts[['Age','Insulin','DiabetesPedigreeFunction']])

X_tr[['Glucose','BMI','BloodPressure','SkinThickness']] = scaler2.fit_transform(X_tr[['Glucose','BMI','BloodPressure','SkinThickness']])
X_ts[['Glucose','BMI','BloodPressure','SkinThickness']] = scaler2.transform(X_ts[['Glucose','BMI','BloodPressure','SkinThickness']])


### Building and fitting 

In [ ]:
model = models.Sequential()

model.add(layers.Dense(20, activation = 'relu', input_shape=(X_tr.shape[1],)))
model.add(layers.Dense(1, activation = 'sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',
             optimizer='sgd',
             metrics=['accuracy'])

In [ ]:
es = EarlyStopping(monitor='val_loss',mode='min', patience=10, verbose=1)

history= model.fit(X_tr,Y_tr,
                 epochs=75,
                 batch_size=15,
                 validation_data=(X_ts,Y_ts),
                 callbacks=[es])

### Evaluation

In [ ]:
fig, ax=plt.subplots(2,1,figsize=(12,10))
fig.suptitle('Train evaluation')

sns.lineplot(ax= ax[0],x=np.arange(0,len(history.history['accuracy'])),y=history.history['accuracy'])
sns.lineplot(ax= ax[0],x=np.arange(0,len(history.history['accuracy'])),y=history.history['val_accuracy'])

ax[0].legend(['Train','Validation'])
ax[0].set_title('Accuracy')

sns.lineplot(ax= ax[1],x=np.arange(0,len(history.history['loss'])),y=history.history['loss'])
sns.lineplot(ax= ax[1],x=np.arange(0,len(history.history['loss'])),y=history.history['val_loss'])

ax[1].legend(['Train','Validation'])
ax[1].set_title('Loss')


plt.show()

In [ ]:
Y_pred = model.predict(X_ts)

error = []
grid = np.linspace(min(Y_pred)[0],max(Y_pred)[0],10000)


for i in grid:
    y_pred_i = (Y_pred>i).astype('int32')
    fails = confusion_matrix(Y_ts,y_pred_i)[0,1]+confusion_matrix(Y_ts,y_pred_i)[1,0]
    error_i = fails/len(Y_ts)
    error.append(error_i)

i_error = list(zip(grid,error))
i_min = i_error[np.argmin(error)][0]


plt.figure(figsize=(12,8))
griderror = sns.lineplot(x=grid, y=error)
griderror.set_title('Error by cutoff')
griderror.set_xlabel('cutoff')
griderror.set_ylabel('error')
plt.scatter(x=i_min,y=min(error), s=300, color='r', marker='+', alpha=0.3)
plt.show()
    

Y_pred=(Y_pred>i_min).astype('int32')
print('Y_pred > {:.4f}, means positive'.format(i_min))

conf_mat = confusion_matrix(Y_ts,Y_pred)
sns.set_style(style='dark')
plt.figure(figsize=(12,8))
heatmap = sns.heatmap(conf_mat,vmin=np.min(conf_mat.all()), vmax=np.max(conf_mat), annot=True,fmt='d', annot_kws={"fontsize":20},cmap='Spectral')
heatmap.set_title('Confusion Matrix Heatmap\n¿Is diabetic?', fontdict={'fontsize':15}, pad=12)
heatmap.set_xlabel('Predicted',fontdict={'fontsize':14})
heatmap.set_ylabel('Actual',fontdict={'fontsize':14})
heatmap.set_xticklabels(['NO','YES'], fontdict={'fontsize':12})
heatmap.set_yticklabels(['NO','YES'], fontdict={'fontsize':12})
plt.show()


print('-Acuracy achieved: {:.2f}%\n-Accuracy by model was: {:.2f}%\n-Accuracy by validation was: {:.2f}%'.
      format((1-min(error))*100,(history.history['accuracy'][-1])*100,(history.history['val_accuracy'][-1])*100))

# Model 2

Same process dropping zero values for columns:
* Blood Pressure
* BMI
* Glucose

In [ ]:
i=[]
i = list(df[df.BloodPressure.isin([0])].index)
i.extend(list(df[df.BMI.isin([0])].index))
i.extend(list(df[df.Glucose.isin([0])].index))

df1 = df.drop(index=i)


X_tr, X_ts, Y_tr, Y_ts = train_test_split(df1.drop(columns=['Outcome']), df1['Outcome'],
                                          test_size=0.15, shuffle=True,random_state=42,stratify=df1['Outcome'])

scaler1 = MinMaxScaler()
scaler2 = StandardScaler()
X_tr[['Age','Insulin','DiabetesPedigreeFunction']] = scaler1.fit_transform(X_tr[['Age','Insulin','DiabetesPedigreeFunction']])
X_ts[['Age','Insulin','DiabetesPedigreeFunction']] = scaler1.transform(X_ts[['Age','Insulin','DiabetesPedigreeFunction']])

X_tr[['Glucose','BMI','BloodPressure','SkinThickness']] = scaler2.fit_transform(X_tr[['Glucose','BMI','BloodPressure','SkinThickness']])
X_ts[['Glucose','BMI','BloodPressure','SkinThickness']] = scaler2.transform(X_ts[['Glucose','BMI','BloodPressure','SkinThickness']])


In [ ]:
model = models.Sequential()

model.add(layers.Dense(20, activation = 'relu', input_shape=(X_tr.shape[1],)))
model.add(layers.Dense(1, activation = 'sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',
             optimizer='sgd',
             metrics=['accuracy'])

In [ ]:
es = EarlyStopping(monitor='val_loss',mode='min', patience=10, verbose=1)

history= model.fit(X_tr,Y_tr,
                 epochs=75,
                 batch_size=15,
                 validation_data=(X_ts,Y_ts),
                 callbacks=[es])

### Evaluation

In [ ]:
fig, ax=plt.subplots(2,1,figsize=(12,10))
fig.suptitle('Train evaluation')

sns.lineplot(ax= ax[0],x=np.arange(0,len(history.history['accuracy'])),y=history.history['accuracy'])
sns.lineplot(ax= ax[0],x=np.arange(0,len(history.history['accuracy'])),y=history.history['val_accuracy'])

ax[0].legend(['Train','Validation'])
ax[0].set_title('Accuracy')

sns.lineplot(ax= ax[1],x=np.arange(0,len(history.history['loss'])),y=history.history['loss'])
sns.lineplot(ax= ax[1],x=np.arange(0,len(history.history['loss'])),y=history.history['val_loss'])

ax[1].legend(['Train','Validation'])
ax[1].set_title('Loss')


plt.show()

In [ ]:
Y_pred = model.predict(X_ts)

error = []
grid = np.linspace(min(Y_pred)[0],max(Y_pred)[0],10000)


for i in grid:
    y_pred_i = (Y_pred>i).astype('int32')
    fails = confusion_matrix(Y_ts,y_pred_i)[0,1]+confusion_matrix(Y_ts,y_pred_i)[1,0]
    error_i = fails/len(Y_ts)
    error.append(error_i)

i_error = list(zip(grid,error))
i_min = i_error[np.argmin(error)][0]


plt.figure(figsize=(12,8))
griderror = sns.lineplot(x=grid, y=error)
griderror.set_title('Error by cutoff')
griderror.set_xlabel('cutoff')
griderror.set_ylabel('error')
plt.scatter(x=i_min,y=min(error), s=300, color='r', marker='+', alpha=0.3)
plt.show()
    

Y_pred=(Y_pred>i_min).astype('int32')
print('Y_pred > {:.4f}, means positive'.format(i_min))

conf_mat = confusion_matrix(Y_ts,Y_pred)
sns.set_style(style='dark')
plt.figure(figsize=(12,8))
heatmap = sns.heatmap(conf_mat,vmin=np.min(conf_mat.all()), vmax=np.max(conf_mat), annot=True,fmt='d', annot_kws={"fontsize":20},cmap='Spectral')
heatmap.set_title('Confusion Matrix Heatmap\n¿Is diabetic?', fontdict={'fontsize':15}, pad=12)
heatmap.set_xlabel('Predicted',fontdict={'fontsize':14})
heatmap.set_ylabel('Actual',fontdict={'fontsize':14})
heatmap.set_xticklabels(['NO','YES'], fontdict={'fontsize':12})
heatmap.set_yticklabels(['NO','YES'], fontdict={'fontsize':12})
plt.show()


print('-Acuracy achieved: {:.2f}%\n-Accuracy by model was: {:.2f}%\n-Accuracy by validation was: {:.2f}%'.
      format((1-min(error))*100,(history.history['accuracy'][-1])*100,(history.history['val_accuracy'][-1])*100))

## Random Forest Classifier Model

In [ ]:
X_tr, X_ts, Y_tr, Y_ts = train_test_split(df.drop(columns='Outcome'), df['Outcome'],
                                          test_size=0.15, shuffle=True,random_state=42,stratify=df['Outcome'])

scaler1 = MinMaxScaler()
scaler2 = StandardScaler()
X_tr[['Age','Insulin','DiabetesPedigreeFunction']] = scaler1.fit_transform(X_tr[['Age','Insulin','DiabetesPedigreeFunction']])
X_ts[['Age','Insulin','DiabetesPedigreeFunction']] = scaler1.transform(X_ts[['Age','Insulin','DiabetesPedigreeFunction']])

X_tr[['Glucose','BMI','BloodPressure','SkinThickness']] = scaler2.fit_transform(X_tr[['Glucose','BMI','BloodPressure','SkinThickness']])
X_ts[['Glucose','BMI','BloodPressure','SkinThickness']] = scaler2.transform(X_ts[['Glucose','BMI','BloodPressure','SkinThickness']])

In [ ]:
RFC = RandomForestClassifier(n_jobs=2,random_state=42)

RFC.fit(X_tr,Y_tr)
Y_pred = RFC.predict(X_ts)

conf_mat = confusion_matrix(Y_ts,Y_pred)
sns.set_style(style='dark')
plt.figure(figsize=(12,8))
heatmap = sns.heatmap(conf_mat,vmin=np.min(conf_mat.all()), vmax=np.max(conf_mat), annot=True,fmt='d', annot_kws={"fontsize":20},cmap='Spectral')
heatmap.set_title('Confusion Matrix Heatmap\nRandom Forest Classifier\n¿Is diabetic?', fontdict={'fontsize':15}, pad=12)
heatmap.set_xlabel('Predicted',fontdict={'fontsize':14})
heatmap.set_ylabel('Actual',fontdict={'fontsize':14})
heatmap.set_xticklabels(['NO','YES'], fontdict={'fontsize':12})
heatmap.set_yticklabels(['NO','YES'], fontdict={'fontsize':12})
plt.show()

print('Accuracy of Random Forest Classifier is: {:.3f}%'.format(accuracy_score(Y_ts,Y_pred)*100))

## Gradient Boosting Classifier Model

In [ ]:
X_tr, X_ts, Y_tr, Y_ts = train_test_split(df.drop(columns='Outcome'), df['Outcome'],
                                          test_size=0.15, shuffle=True,random_state=42,stratify=df['Outcome'])

scaler1 = MinMaxScaler()
scaler2 = StandardScaler()
X_tr[['Age','Insulin','DiabetesPedigreeFunction']] = scaler1.fit_transform(X_tr[['Age','Insulin','DiabetesPedigreeFunction']])
X_ts[['Age','Insulin','DiabetesPedigreeFunction']] = scaler1.transform(X_ts[['Age','Insulin','DiabetesPedigreeFunction']])

X_tr[['Glucose','BMI','BloodPressure','SkinThickness']] = scaler2.fit_transform(X_tr[['Glucose','BMI','BloodPressure','SkinThickness']])
X_ts[['Glucose','BMI','BloodPressure','SkinThickness']] = scaler2.transform(X_ts[['Glucose','BMI','BloodPressure','SkinThickness']])

In [ ]:
GBC = GradientBoostingClassifier(random_state=42)

GBC.fit(X_tr,Y_tr)
Y_pred = GBC.predict(X_ts)

conf_mat = confusion_matrix(Y_ts,Y_pred)
sns.set_style(style='dark')
plt.figure(figsize=(12,8))
heatmap = sns.heatmap(conf_mat,vmin=np.min(conf_mat.all()), vmax=np.max(conf_mat), annot=True,fmt='d', annot_kws={"fontsize":20},cmap='Spectral')
heatmap.set_title('Confusion Matrix Heatmap\nGradien Boosting Classifier\n¿Is diabetic?', fontdict={'fontsize':15}, pad=12)
heatmap.set_xlabel('Predicted',fontdict={'fontsize':14})
heatmap.set_ylabel('Actual',fontdict={'fontsize':14})
heatmap.set_xticklabels(['NO','YES'], fontdict={'fontsize':12})
heatmap.set_yticklabels(['NO','YES'], fontdict={'fontsize':12})
plt.show()

print('Accuracy of Gradient Boosting Classifier is: {:.3f}%'.format(accuracy_score(Y_ts,Y_pred)*100))